## Pipeline Tutorial with HeteroSecureBoost

### install

`Pipeline` is distributed along with [fate_client](https://pypi.org/project/fate-client/).

```bash
pip install fate_client
```

To use Pipeline, we need to first specify which `FATE Flow Service` to connect to. Once `fate_client` installed, one can find an cmd enterpoint name `pipeline`:

In [3]:
import sys
print(sys.version)

3.8.13 (default, Oct 21 2022, 23:50:54) 
[GCC 11.2.0]


In [4]:
!python --version

Python 3.8.13


In [5]:
!pipeline --help

Usage: pipeline [OPTIONS] COMMAND [ARGS]...

Options:
  --help  Show this message and exit.

Commands:
  config  pipeline config tool
  init    - DESCRIPTION: Pipeline Config Command.


Assume we have a `FATE Flow Service` in 127.0.0.1:9380(defaults in standalone), then exec

In [6]:
!pipeline init --ip 127.0.0.1 --port 9380

Pipeline configuration succeeded.


### Hetero SecureBoost Example

 Before start a modeling task, data to be used should be uploaded. Please refer to this [guide](https://github.com/FederatedAI/FATE/blob/master/doc/tutorial/pipeline/pipeline_tutorial_upload.ipynb).

The `pipeline` package provides components to compose a `FATE pipeline`.

In [7]:
from pipeline.backend.pipeline import PipeLine
from pipeline.component import Reader, DataTransform, Intersection, HeteroSecureBoost, Evaluation
from pipeline.interface import Data

Make a `pipeline` instance:

    - initiator: 
        * role: guest
        * party: 9999
    - roles:
        * guest: 9999
        * host: 10000
    

In [8]:
pipeline = PipeLine() \
        .set_initiator(role='guest', party_id=9999) \
        .set_roles(guest=9999, host=10000)

Now we need to upload our data.

In [9]:
import os

data_base = os.path.join(os.getcwd(), "../data/")
partition = 4
guest = 9999

dense_data = {"name": "breast_hetero_guest", "namespace": f"experiment"}
tag_data = {"name": "breast_hetero_host", "namespace": f"experiment"}

In [10]:
#pipeline_upload = PipeLine().set_initiator(role="guest", party_id=guest).set_roles(guest=guest)
pipeline_upload = PipeLine().set_initiator(role='guest', party_id=9999).set_roles(guest=9999)

In [11]:
pipeline_upload.add_upload_data(file=os.path.join(data_base, "breast_hetero_guest.csv"),
                                table_name=dense_data["name"],             # table name
                                namespace=dense_data["namespace"],         # namespace
                                head=1, partition=partition)               # data info


pipeline_upload.add_upload_data(file=os.path.join(data_base, "breast_hetero_host.csv"),
                                table_name=tag_data["name"],
                                namespace=tag_data["namespace"],
                                head=1, partition=partition)

In [12]:
pipeline_upload.upload(drop=1)

 UPLOADING:||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||100.00%

2023-05-30 08:16:52.723 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202305301316525295410



2023-05-30 08:16:52.730 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00
2023-05-30 08:16:53.738 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:01
m2023-05-30 08:16:54.750 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-05-30 08:16:54.751 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:02
2023-05-30 08:16:55.762 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:03
2023-05-30 08:16:56.785 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:04
2023-05-30 08:16:57.821 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:05
2023-05-30 08:16:58.842 |

 UPLOADING:||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||100.00%

2023-05-30 08:16:58.978 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202305301316588534190

2023-05-30 08:16:58.984 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00


2023-05-30 08:17:00.000 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:01
m2023-05-30 08:17:01.013 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-05-30 08:17:01.014 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:02
2023-05-30 08:17:02.030 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:03
2023-05-30 08:17:03.038 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:89 - Job is success!!! Job id is 202305301316588534190
2023-05-30 08:17:03.038 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:90 - Total time: 0:00:04


Define a `Reader` to load data

In [13]:
reader_0 = Reader(name="reader_0")
# set guest parameter
reader_0.get_party_instance(role='guest', party_id=9999).component_param(
    table={"name": "breast_hetero_guest", "namespace": "experiment"})
# set host parameter
reader_0.get_party_instance(role='host', party_id=10000).component_param(
    table={"name": "breast_hetero_host", "namespace": "experiment"})

Add a `DataTransform` component to parse raw data into Data Instance

In [14]:
data_transform_0 = DataTransform(name="data_transform_0")
# set guest parameter
data_transform_0.get_party_instance(role='guest', party_id=9999).component_param(
    with_label=True)
data_transform_0.get_party_instance(role='host', party_id=[10000]).component_param(
    with_label=False)

Add a `Intersection` component to perform PSI for hetero-scenario

In [15]:
intersect_0 = Intersection(name="intersect_0")

Now, we define the `HeteroSecureBoost` component. The following parameters will be set for all parties involved.

In [16]:
hetero_secureboost_0 = HeteroSecureBoost(name="hetero_secureboost_0",
                                         num_trees=5,
                                         bin_num=16,
                                         task_type="classification",
                                         objective_param={"objective": "cross_entropy"},
                                         encrypt_param={"method": "paillier"},
                                         tree_param={"max_depth": 3})


To show the evaluation result, an "Evaluation" component is needed.

In [17]:
evaluation_0 = Evaluation(name="evaluation_0", eval_type="binary")

Add components to pipeline, in order of execution:

    - data_transform_0 comsume reader_0's output data
    - intersect_0 comsume data_transform_0's output data
    - hetero_secureboost_0 consume intersect_0's output data
    - evaluation_0 consume hetero_secureboost_0's prediciton result on training data

Then compile our pipeline to make it ready for submission.

In [18]:
pipeline.add_component(reader_0)
pipeline.add_component(data_transform_0, data=Data(data=reader_0.output.data))
pipeline.add_component(intersect_0, data=Data(data=data_transform_0.output.data))
pipeline.add_component(hetero_secureboost_0, data=Data(train_data=intersect_0.output.data))
pipeline.add_component(evaluation_0, data=Data(data=hetero_secureboost_0.output.data))
pipeline.compile();


Now, submit(fit) our pipeline:

In [19]:
pipeline.fit()


2023-05-30 08:17:43.523 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202305301317407628510

2023-05-30 08:17:43.530 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00
m2023-05-30 08:17:44.541 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-05-30 08:17:44.542 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:01
2023-05-30 08:17:45.555 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:02
m2023-05-30 08:17:46.572 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-05-30 08:17:46.573 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:03
2023-05-30 08:17:47.585 | INFO     | pipeline.utils.invoker.job_submitter:

Once training is done, trained model may be used for prediction. Optionally, save the trained pipeline for future use.

In [20]:
pipeline.dump("pipeline_saved.pkl");

First, deploy needed components from train pipeline

In [22]:
import pickle

In [26]:
pipeline = PipeLine.load_model_from_file('pipeline_saved.pkl')
pipeline.deploy_component([pipeline.data_transform_0, pipeline.intersect_0, pipeline.hetero_secureboost_0]);

In [29]:
print(dir(pipeline.data_transform_0))

['_Component__instance', '_Component__party_instance', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slotnames__', '__str__', '__subclasshook__', '__weakref__', '_component_name', '_component_param', '_component_parameter_keywords', '_decrease_instance_count', '_get_all_party_instance', '_greater_equal_than', '_in', '_less_equal_than', '_module_name', '_not_in', '_provider', '_provider_version', '_range', '_role_parameter_keywords', '_source_provider', '_validate_param', 'check', 'check_and_change_lower', 'check_boolean', 'check_decimal_float', 'check_defined_type', 'check_nonnegative_number', 'check_open_unit_interval', 'check_positive_integer', 'check_positive_number', 'check_string', 'check_valid_value', 'component_param', 'data_typ

Define new `Reader` components for reading prediction data

In [14]:
reader_1 = Reader(name="reader_1")
reader_1.get_party_instance(role="guest", party_id=9999).component_param(table={"name": "breast_hetero_guest", "namespace": "experiment"})
reader_1.get_party_instance(role="host", party_id=10000).component_param(table={"name": "breast_hetero_host", "namespace": "experiment"})

Optionally, define new `Evaluation` component.

In [15]:
evaluation_0 = Evaluation(name="evaluation_0", eval_type="binary")

Add components to predict pipeline in order of execution:

In [16]:
predict_pipeline = PipeLine()
predict_pipeline.add_component(reader_1)\
                .add_component(pipeline, 
                               data=Data(predict_input={pipeline.data_transform_0.input.data: reader_1.output.data}))\
                .add_component(evaluation_0, data=Data(data=pipeline.hetero_secureboost_0.output.data));


Then, run prediction job

In [17]:
predict_pipeline.predict()

2021-12-31 03:25:35.541 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:123 - Job id is 202112310325328444510

2021-12-31 03:25:47.384 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:144 - Job is still waiting, time elapse: 0:00:11
m2021-12-31 03:25:47.903 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:173 - 
2021-12-31 03:25:52.078 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component reader_1, time elapse: 0:00:16
m2021-12-31 03:25:54.161 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:173 - 
2021-12-31 03:25:58.545 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component data_transform_0, time elapse: 0:00:23
m2021-12-31 03:26:01.167 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:173 - 
2021-12-31 03:26:07.502 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component in

For more demo on using pipeline to submit jobs, please refer to [pipeline demos](https://github.com/FederatedAI/FATE/tree/master/examples/pipeline/demo)